In [1]:
import ROOT as R
import uproot

import import_ipynb
import setPath
from os import listdir
from os.path import isfile, join
from Input.OpenDataPandaFramework13TeV import *

import time

import pandas as pd
import numpy as np

import pyStats

%jsroot on

R.gInterpreter.ProcessLine('#include "Analysis/AnalysisSelector.cpp"')


Welcome to JupyROOT 6.24/02
importing Jupyter notebook from setPath.ipynb
importing Jupyter notebook from /storage/galaxy/jobs_directory/005/5239/working/jupyter/Input/OpenDataPandaFramework13TeV.ipynb
This library contains handy functions to ease the access and use of the 13TeV ATLAS OpenData release

getBkgCategories()
	 Dumps the name of the various background cataegories available 
	 as well as the number of samples contained in each category.
	 Returns a vector with the name of the categories

getSamplesInCategory(cat)
	 Dumps the name of the samples contained in a given category (cat)
	 Returns dictionary with keys being DSIDs and values physics process name from filename.

getMCCategory()
	 Returns dictionary with keys DSID and values MC category

initialize(indir)
	 Collects all the root files available in a certain directory (indir)

getSkims(indir)
	 Prints all available skims in the directory



Setting luminosity to 10064 pb^-1

###############################
#### Backgrou

0

In [2]:
opendatadir = "/storage/shared/data/fys5555/ATLAS_opendata/"
analysis = "GamGam"

In [3]:
# Manipulate the simulated Monte Carlo data and choose which background and signal samples you want to use
mcfiles = initialize(opendatadir+"/"+analysis+"/MC")
datafiles = initialize(opendatadir+"/"+analysis+"/Data")
allfiles = z = {**mcfiles, **datafiles}
Backgrounds = getBkgCategories()

Backgrounds = ['Higgs']

####################################################################################################
SIGNAL SAMPLES
####################################################################################################
WARNING 	 File for SlepSlep_direct_200p5_1p0_2L8 not found in /storage/shared/data/fys5555/ATLAS_opendata//GamGam/MC
WARNING 	 File for SlepSlep_direct_100p5_1p0_2L8 not found in /storage/shared/data/fys5555/ATLAS_opendata//GamGam/MC
WARNING 	 File for SlepSlep_direct_500p5_1p0_2L8 not found in /storage/shared/data/fys5555/ATLAS_opendata//GamGam/MC
WARNING 	 File for SlepSlep_direct_300p5_1p0_2L8 not found in /storage/shared/data/fys5555/ATLAS_opendata//GamGam/MC
WARNING 	 File for SlepSlep_direct_100p0_50p0_2L8 not found in /storage/shared/data/fys5555/ATLAS_opendata//GamGam/MC
WARNING 	 File for SlepSlep_direct_500p0_100p0_2L8 not found in /storage/shared/data/fys5555/ATLAS_opendata//GamGam/MC
WARNING 	 File for SlepSlep_direct_200p0_100p0_2L8 not found in /storage/share

In [4]:
files = []
for Category in Backgrounds:
    Type = mcfiles[Category]['type']
    for File in mcfiles[Category]['files']:
        files.append(File)
        
for File in datafiles['data']['files']:
    files.append(File)

In [5]:
chain = R.TChain('mini') 

for File in files:
        chain.Add(File) 

In [6]:
if not os.path.exists('./Histograms'):
    os.makedirs('./Histograms')
if not os.path.exists('./Histograms/MC/'):
    os.makedirs('./Histograms/MC')
if not os.path.exists('./Histograms/Data/'):
    os.makedirs('./Histograms/Data')

In [7]:
selection = R.AnalysisSelector(chain, analysis)

Process the chains for both the Monte Carlo and Data at the same time, the relevant histograms and features for machine learning can be aquired by calling the TChain Process class from the SelectorProxy class via the Selector function

- R.SelectorProxy().Selector().func()

The relevant functions are

- .GetHistogram((string) Category)

In [8]:
%%time

selection.Process('Higgs')

CPU times: user 11.1 s, sys: 568 ms, total: 11.7 s
Wall time: 11.8 s
-------------------------------------------
Processing MC and Data
Number of events to process: 10271759
-------------------------------------------
 FCN=14.6648 FROM HESSE     STATUS=NOT POSDEF     23 CALLS         158 TOTAL
                     EDM=1.86995e-13    STRATEGY= 1      ERR MATRIX NOT POS-DEF
  EXT PARAMETER                APPROXIMATE        STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.13290e+05   9.01067e+01   2.70106e-02   1.78750e-09
   2  p1          -2.11700e+03   9.73198e-01   5.04732e-04  -3.07233e-07
   3  p2           1.36698e+01   6.70524e-03   3.25913e-06  -2.44238e-05
   4  p3          -3.01078e-02   3.33820e-05   7.17826e-09   6.71368e-03
   5  p4           1.19100e+02     fixed    
   6  p5           1.25000e+02     fixed    
   7  p6           2.39000e+00     fixed    
Total number of processed events: 10271759
Number of e

Warning in <TROOT::Append>: Replacing existing TH1: h_myy (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_myy (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_myy (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_myy (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_myy (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_myy (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_myy (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_myy (Potential memory leak).


## Setup the plots

In [9]:
colours = {}

colours["Diboson"] = R.kGreen; 
colours["Zjets"] = R.kYellow; 
colours["ttbar"] = R.kRed;
colours["singleTop"] = R.kBlue-7; 
colours["Wjets"] = R.kBlue+3; 
colours["topX"] = R.kOrange+1; 
colours["Higgs"] = R.kMagenta; 
colours["Wjetsincl"] = R.kBlue-10;
colours["Zjetsincl"] = R.kYellow-9;

### Get histograms from C++ class

In [10]:
H_myy = {}

for bkg in Backgrounds:
    H_myy[bkg] = R.TH1D()
    
    H_myy[bkg].SetNameTitle('H_myy', 'Invariant mass')
    H_myy[bkg].SetBins(24, 105, 160)
    
    H_myy[bkg].Reset()
    
    H_myy[bkg].Add(selection.Selector().GetHistogram('myy', bkg)) # Aquire the specific histogram from the Selector pointer from the SelectorProxy class    
    

In [11]:
H_myy_d = R.TH1D()

H_myy_d.SetNameTitle('H_myy', 'Invariant mass')
H_myy_d.SetBins(24, 105, 160)

H_myy_d.Reset()

H_myy_d.Add(selection.Selector().GetHistogram('myy', 'data'))


True

### Create the plots

In [12]:
H_myy['Higgs'].SetLineColor(R.kBlack); 


In [13]:
stack_myy = R.THStack("Invariant mass", "");

for bkg in Backgrounds: 
    stack_myy.RecursiveRemove(H_myy[bkg]); ## Remove previously stacked histograms  
    
    stack_myy.Add(H_myy[bkg]); 
    

In [14]:
H_myy_d.SetLineColor(R.kBlack); 
H_myy_d.SetMarkerStyle(R.kFullCircle); 
H_myy_d.SetMarkerColor(R.kBlack); 

In [15]:
fit = selection.Selector().GetFit('bkg+sig')
bkg = selection.Selector().GetFit('bkg')

In [23]:
R.gROOT.SetStyle("ATLAS")

C = R.TCanvas("c", "c", 700, 750)

upper_pad = R.TPad("upper_pad", "", 0, 0.35, 1, 1)
lower_pad = R.TPad("lower_pad", "", 0, 0, 1, 0.35)
for p in [upper_pad, lower_pad]:
    p.SetLeftMargin(0.14)
    p.SetRightMargin(0.05)
    p.SetTickx(False)
    p.SetTicky(False)
upper_pad.SetBottomMargin(0)
lower_pad.SetTopMargin(0)
lower_pad.SetBottomMargin(0.3)
 
upper_pad.Draw()
lower_pad.Draw()

upper_pad.cd()

stack_myy.Draw("hist"); 
stack_myy.SetMaximum(8e3); 
stack_myy.SetMinimum(1); 
stack_myy.GetYaxis().SetTitle("# events");
stack_myy.GetYaxis().SetTitleOffset(1.3); 
stack_myy.GetXaxis().SetTitle("m_{#gamma#gamma} (GeV)");
stack_myy.GetXaxis().SetTitleOffset(1.3);
H_myy_d.Draw("same E"); 

fit.SetLineColor(R.kRed)
fit.SetLineStyle(1)
fit.SetLineWidth(2)
fit.Draw('Same')

bkg.SetLineColor(R.kBlue)
bkg.SetLineStyle(2)
bkg.SetLineWidth(2)
bkg.Draw("SAME")

R.gStyle.SetLegendBorderSize(0); ## Remove (default) border around legend 
leg = R.TLegend(0.65, 0.60, 0.9, 0.85); 

leg.SetTextFont(42)
leg.SetFillStyle(0)
leg.SetBorderSize(0)
leg.SetTextSize(0.05)
leg.SetTextAlign(32)

leg.Clear();
#for bkg in Backgrounds: 
#    leg.AddEntry(H_myy[bkg], bkg, "f")  ## Add your histograms to the legend
leg.AddEntry(H_myy_d, "Data", "lep") 
leg.AddEntry(fit, "Signal + Bkg", "l")
leg.AddEntry(bkg, "Background", "l")
leg.AddEntry(H_myy['Higgs'], "Signal", "l")

leg.Draw()

lower_pad.cd()

ratiobkg = R.TH1I("zero", "", 100, 105, 160)
ratiobkg.SetLineColor(R.kBlue)
ratiobkg.SetLineStyle(2)
ratiobkg.SetLineWidth(2)
ratiobkg.SetMinimum(-125)
ratiobkg.SetMaximum(250)
ratiobkg.GetXaxis().SetLabelSize(0.08)
ratiobkg.GetXaxis().SetTitleSize(0.12)
ratiobkg.GetXaxis().SetTitleOffset(1.0)
ratiobkg.GetYaxis().SetLabelSize(0.08)
ratiobkg.GetYaxis().SetTitleSize(0.09)
ratiobkg.GetYaxis().SetTitle("Data - Bkg")
ratiobkg.GetYaxis().CenterTitle()
ratiobkg.GetYaxis().SetTitleOffset(0.7)
ratiobkg.GetYaxis().SetNdivisions(503, False)
ratiobkg.GetYaxis().ChangeLabel(-1, -1, 0)
ratiobkg.GetXaxis().SetTitle("m_{#gamma#gamma} [GeV]")
ratiobkg.Draw("AXIS")

line = R.TLine(105, 0, 160, 0)
line.SetLineWidth(2)
line.SetLineStyle(2)
line.SetLineColor(R.kBlue)
line.Draw()

ratiosig = R.TH1F("ratiosig", "ratiosig", 5500, 105, 160)
ratiosig.Eval(fit)
ratiosig.SetLineColor(2)
ratiosig.SetLineStyle(1)
ratiosig.SetLineWidth(2)
ratiosig.Add(bkg, -1)
ratiosig.Draw("SAME")
 
ratiodata = H_myy_d.Clone()
ratiodata.Add(bkg, -1)
for i in range(1, H_myy_d.GetNbinsX()):
    ratiodata.SetBinError(i, H_myy_d.GetBinError(i))
ratiodata.Draw("E SAME")

C.Draw();

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Warning in <TROOT::Append>: Replacing existing TH1: zero (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: ratiosig (Potential memory leak).


In [31]:
C = R.TCanvas("c", "c", 700, 750)

w = R.RooWorkspace('w')

g = w.factory('Gaussian::g(x[122,128],mean[123,126],sigma[0.1,50])')

x = w.var('x')
sigma = w.var('sigma')
mean = w.var('mean')

data = R.RooDataHist("data", "data", x, H_myy_d)

nll = g.createNLL(data)

minimizer = R.RooMinimizer(nll)
minimizer.minimize('Minuit2')

pll_mean = nll.createProfile(mean)

frame = mean.frame()

tmp = nll.plotOn(frame, R.RooFit.ShiftToZero())
p = tmp.getObject(0)

frame.Draw()

frame.SetMaximum(10)
frame.GetXaxis().SetRange(118,130)
C.Draw()

[#1] INFO:DataHandling -- RooDataHist::adjustBinning(data): fit range of variable x expanded to nearest bin boundaries: [122,128] --> [121.042,130.208]
[#1] INFO:Minization -- createNLL: caching constraint set under name CONSTR_OF_PDF_g_FOR_OBS_x with 0 entries
 **********
 **   37 **SET PRINT           1
 **********
 **********
 **   38 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 mean         1.24500e+02  3.00000e-01    1.23000e+02  1.26000e+02
     2 sigma        2.50500e+01  4.99000e+00    1.00000e-01  5.00000e+01
 **********
 **   39 **SET ERR         0.5
 **********
 **********
 **   40 **SET PRINT           1
 **********
 **********
 **   41 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY
 **********
 **   42 **MIGRAD        1000           1
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
 START MIGRAD MINIMIZATION.  STRATEGY  1.  CONVE

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Warning in <ROOT::Math::FitConfig::CreateMinimizer>: Could not create the Minuit2 minimizer. Try using the minimizer Minuit


In [18]:
print(mean.getValV(), mean.getError())

124.30357365354124 0.1712991837656972


In [19]:
# turn fit into histogram with equal amount of bins to data histogram
bkg.SetNpx(24)
a = bkg.GetHistogram()

b = a.GetBinContent(8)
b += a.GetBinContent(9)
b += a.GetBinContent(10)
n_obs = int(H_myy_d.GetBinContent(8))
n_obs += int(H_myy_d.GetBinContent(9))
n_obs += int(H_myy_d.GetBinContent(10))

print(n_obs, b)

Z = np.sqrt(2*(n_obs*np.log(n_obs/b) - n_obs + b))
Z

10913 10554.966038788203


3.4655152703497176

In [26]:
Z = pyStats.countingExperimentWithBkgCorr(name='yy', intLum=10.06, intLumUnc=0.37)

sig_eff = selection.Selector().GetSignalEfficiency()

# Signal region
n_obs = 0
b = 0
# Select bins corresponding to signal region
for i in range(8,11):
    n_obs += int(H_myy_d.GetBinContent(i))
    b += a.GetBinContent(i)
Z.addChannel('Signal region', bkg=b, bkgUncUncorr=0, bkgUncCorr=0, Nobs=n_obs, eff=sig_eff)

# Control region
n_obs = 0
b = 0
for i in range(1,6):
    n_obs += int(H_myy_d.GetBinContent(i))
    b += a.GetBinContent(i)
Z.addChannel('Control region', bkg=b, bkgUncUncorr=0, bkgUncCorr=0, Nobs=n_obs, eff=0)
Z.setPriors(bkg='gauss');

print(Z)

print('Significance =', Z.getSignificance())

---------------------------
Counting experiment "yy"
---------------------------
Int. luminosity = 10.06 +/- 0.37
---------------------------
Channel "Signal region":
   Background = 10554.966038788167 +/- 0(uncorr) +/- 0(corr)
   Observed events = 10913
   Signal efficiency = 0.6122264266014099 +/- 0.0
Channel "Control region":
   Background = 28220.41138492364 +/- 0(uncorr) +/- 0(corr)
   Observed events = 28222
   Signal efficiency = 0 +/- 0.0

Significance = 2.9574820614626876


In [21]:
Z.getBayesianLimit()

91.38486691887522

In [22]:
"""m2sig,m1sig,expected,p1sig,p2sig = Z.getBayesianExpectedLimit();
print( "\nExpected limit and bands:" );
print( "  -2sigma                 -1sigma                 median                +1sigma                +2sigma" );
print( m2sig, "   ", m1sig, "   ", expected, "   ", p1sig, "   ", p2sig );"""

'm2sig,m1sig,expected,p1sig,p2sig = Z.getBayesianExpectedLimit();\nprint( "\nExpected limit and bands:" );\nprint( "  -2sigma                 -1sigma                 median                +1sigma                +2sigma" );\nprint( m2sig, "   ", m1sig, "   ", expected, "   ", p1sig, "   ", p2sig );'